## Step 1: 環境設置

In [10]:
import os
import sys
from pathlib import Path
from dotenv import dotenv_values
from contextlib import AsyncExitStack
from datetime import datetime

# 設定 Python 路徑，以便導入專案模組
project_root = Path.cwd().parent.parent.parent
sys.path.insert(0, str(project_root / "backend" / "src"))

# 找到 .env 檔案
env_file = project_root / "backend" / ".env"
if not env_file.exists():
    env_file = Path.home() / "Documents/workspace/CasualTrader/backend/.env"

# 讀取並設定環境變數
if env_file.exists():
    env_vars = dotenv_values(env_file)
    for key, value in env_vars.items():
        os.environ[key] = value.strip('"').strip("'") if value else ""
    print(f"✓ .env 已載入: {env_file}")
else:
    print(f"✗ .env 檔案未找到: {env_file}")

# 驗證必要的環境變數
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")
print(f"✓ PERPLEXITY_API_KEY: {'已設定' if PERPLEXITY_API_KEY else '未設定'}")


✓ .env 已載入: /Users/sacahan/Documents/workspace/CasualTrader/backend/.env
✓ PERPLEXITY_API_KEY: 已設定


## Step 2: 配置 Perplexity MCP

In [11]:
import logging
from agents.mcp import MCPServerStdio
from contextlib import AsyncExitStack
import asyncio

# 設置日誌級別，隱藏 DEBUG 訊息
logging.basicConfig(level=logging.WARNING)
logging.getLogger("agents").setLevel(logging.WARNING)
logging.getLogger("asyncio").setLevel(logging.WARNING)

# 全域變數用於存儲 MCP 伺服器實例和 exit stack
perplexity_mcp = None
_exit_stack = AsyncExitStack()

async def _init_perplexity_mcp_internal():
    """
    內部初始化函式，在異步上下文中執行
    """
    global perplexity_mcp
    
    try:
        print("📦 初始化 Perplexity MCP...")
        
        perplexity_mcp = await _exit_stack.enter_async_context(
            MCPServerStdio(
                name="perplexity_mcp",
                params={
                    "command": "npx",
                    "args": ["-y", "@perplexity-ai/mcp-server"],
                    "env": {"PERPLEXITY_API_KEY": f"{PERPLEXITY_API_KEY}"},
                },
                client_session_timeout_seconds=30.0,
            )
        )

        print("✓ Perplexity MCP 初始化完成")
        return perplexity_mcp

    except Exception as e:
        print(f"✗ 初始化失敗: {type(e).__name__}: {str(e)}")
        raise


async def _list_tools_internal():
    """
    內部工具列表函式
    """
    global perplexity_mcp
    
    try:
        if not perplexity_mcp:
            print("✗ Perplexity MCP 未初始化")
            return []

        # 列出可用工具
        tools_result = await perplexity_mcp.session.list_tools()
        tools = tools_result.tools if hasattr(tools_result, "tools") else []

        print("\n📋 Perplexity MCP 可用工具:")
        print("-" * 60)
        for tool in tools:
            print(f"  • {tool.name}")
            if hasattr(tool, "description"):
                print(f"    描述: {tool.description}")

        return tools

    except Exception as e:
        print(f"✗ 列出工具失敗: {type(e).__name__}: {str(e)}")
        return []


# 執行初始化
print("=" * 60)
print("正在初始化 Perplexity MCP...")
print("=" * 60)

# 使用 await 在 Jupyter 中執行異步操作
perplexity_mcp = await _init_perplexity_mcp_internal()
tools = await _list_tools_internal()
print(f"\n✓ perplexity_mcp 已全域設定: {perplexity_mcp}")

正在初始化 Perplexity MCP...
📦 初始化 Perplexity MCP...
✓ Perplexity MCP 初始化完成

📋 Perplexity MCP 可用工具:
------------------------------------------------------------
  • perplexity_ask
    描述: Engages in a conversation using the Sonar API. Accepts an array of messages (each with a role and content) and returns a ask completion response from the Perplexity model.
  • perplexity_research
    描述: Performs deep research using the Perplexity API. Accepts an array of messages (each with a role and content) and returns a comprehensive research response with citations.
  • perplexity_reason
    描述: Performs reasoning tasks using the Perplexity API. Accepts an array of messages (each with a role and content) and returns a well-reasoned response using the sonar-reasoning-pro model.
  • perplexity_search
    描述: Performs web search using the Perplexity Search API. Returns ranked search results with titles, URLs, snippets, and metadata. Perfect for finding up-to-date facts, news, or specific information.

✓

## Step 3: 建立 Agent

In [12]:
from agents import (
    Agent,
    Runner,
    gen_trace_id,
    trace,
)

# 建立 Agent
agent = Agent(
    name="FinancialAnalyst",
    model="gpt-4o-mini",
    mcp_servers=[perplexity_mcp],
    instructions="你是股市財經研究專家。當用戶要求財經訊息時，你會使用 perplexity_mcp 進行網路搜尋，然後分析並總結結果。",
)

print('✓ Agent 建立成功')

✓ Agent 建立成功


## 測試 1: 股票市場信息搜尋

In [ ]:
async def test_1():
    print('='*70)
    print('測試 1: 執行股票市場搜尋')
    print('='*70)

    prompt = f'搜尋今日({datetime.now().strftime("%Y-%m-%d")})的台股市場表現和主要趨勢，並簡要分析。'

    print(f'\n用戶提示: {prompt}')
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="perplexity_agent_lab", trace_id=trace_id):
            result = await Runner.run(agent, prompt)
            print(f'結果:\n{result.final_output}')
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")

await test_1()

## 測試 2: 技術新聞搜尋

In [ ]:
async def test_2():
    print('\n' + '='*70)
    print('測試 2: 執行科技新聞搜尋')
    print('='*70)
    
    prompt = f"搜尋關於台股上市櫃公司中相關AI題材股最新發布的新聞，並總結主要進展。({datetime.now().strftime('%Y-%m-%d')})"
    
    print(f'\n用戶提示: {prompt}')
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="perplexity_agent_lab", trace_id=trace_id):
            result = await Runner.run(agent, prompt)
            print(f"結果:\n{result.final_output}")
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")

await test_2()

## 測試 3: 經濟數據搜尋

In [13]:
async def test_3():
    print('\n' + '='*70)
    print('測試 3: 執行經濟數據搜尋')
    print('='*70)

    prompt = f"搜尋最新 ({datetime.now().strftime('%Y-%m-%d')}) 的台灣就業數據、GDP 增長率和通脹率，並提供：1. 當前的主要經濟指標，2. 最近的變化趨勢，3. 對市場的潛在影響"

    print(f'\n用戶提示: {prompt}')
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="perplexity_agent_lab", trace_id=trace_id):
            result = await Runner.run(agent, prompt)
            print(f"結果:\n{result.final_output}")
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")

await test_3()


測試 3: 執行經濟數據搜尋

用戶提示: 搜尋最新 (2025-11-19) 的台灣就業數據、GDP 增長率和通脹率，並提供：1. 當前的主要經濟指標，2. 最近的變化趨勢，3. 對市場的潛在影響
結果:
### 台灣最新經濟指標 (截至2025年11月19日)

#### 1. 當前的主要經濟指標
- **就業數據**：
  - **失業率**：3.35%（2025年9月），保持不變，為4個月內的最高水平。失業人數約407,000人，勞動參與率為59.51%。
- **GDP增長率**：
  - **季度增長率**：第三季度GDP年增長率為7.64%，超出市場預期。而環比增長為1.31%。這是連續第十個季度的經濟增長。
- **通脹率**：
  - **消費者物價指數增長率**：2025年10月通脹率為1.48%，較9月上升（1.25%）。這一增長主要受住房和教育娛樂品價格上升影響。

#### 2. 最近的變化趨勢
- **就業**：
  - 失業率於9月持平，與8月相比，青年失業率在此期間有所下降至12.01%。
  
- **GDP**：
  - 自第一季度的5.48%增長後，第二季度和第三季度分別實現了7.96%及7.64%的高增長，顯示出強勁的經濟回暖趨勢，主要由外部需求驅動。

- **通脹**：
  - 2025年通脹率維持在低位，反映出經濟的穩定性。10月的增幅相比於9月顯著，主要因食物和住宅價格上漲，顯示出物價壓力逐漸回升。

#### 3. 對市場的潛在影響
- **對勞動市場**：
  - 低失業率和穩定的勞動參與率可能促進消費者信心，提高私人消費，進一步推動經濟增長。

- **對經濟增長**：
  - 超過預期的GDP增長可能促使政府考慮更積極的經濟政策，並持續投資於高科技和製造業，尤其是在AI和半導體領域。

- **對通脹壓力**：
  - 若通脹持續上升，可能促使中央銀行考慮利率調整，這將影響信貸成本和消費者支出，進而對整體經濟造成連鎖反應。

這些經濟指標反映出台灣在全球經濟復甦中的良好勢頭，並顯示出政府在穩定經濟與控制通脹之間的微妙平衡。
結果:
### 台灣最新經濟指標 (截至2025年11月19日)

#### 1. 當前的主要經濟指標
- **就業數據**：
  - **失業率**：3.35%（2025年9月），保持不